In [1]:
%matplotlib widget

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [ ]:
basedir = Path("/allen/programs/braintv/workgroups/nc-ophys/danielk/deepinterpolation/experiments")
experiment_ids = [785569470, 1048483611, 1048483613, 1048483616, 785569447,
                  788422859, 795901850, 788422825, 795897800, 850517348,
                  951980473, 951980484, 795901895, 806862946, 803965468,
                  806928824]
metadata_files = [basedir / f"ophys_experiment_{i}" / "metadata.csv" for i in experiment_ids]

### Available Experiments

In [ ]:
frames = pd.concat([pd.read_csv(file, header=None).set_index(0).T for file in metadata_files]).set_index("ophys_experiment_id")
frames["depth"] = frames["depth"].astype(int)
frames = frames.sort_values(by=["code", "rig", "depth"]).reset_index()
display(frames)

In [ ]:
df_per_row = [pd.DataFrame(i[1]).T for i in frames.iterrows()]
col_widths = [max([len(f"{i}") for i in frames[c].values]) for c in frames.columns]
options = ["".join([f"{i:{w + 4}}" for i, w in zip(k.values[0], col_widths)]) for k in df_per_row]
eids = [int(i["ophys_experiment_id"].values[0]) for i in df_per_row]
values = [basedir / f"ophys_experiment_{i}" for i in eids]
experiment_selector = widgets.Dropdown(
    options=[(o, v) for o, v in zip(options, values)],
    layout={'width': 'max-content'}, # If the items' names are long
    description='Select experiment:',
    disabled=False
)
display(experiment_selector)

In [ ]:
def get_max_ave_projections(result_dir: Path):
    nmax = result_dir / "noised_maxp.png"
    navg = result_dir / "noised_avgp.png"
    dnmax = result_dir / "denoised_maxp.png"
    dnavg = result_dir / "denoised_avgp.png"

    nmaxim = np.array(Image.open(nmax))
    navgim = np.array(Image.open(navg))
    dnmaxim = np.array(Image.open(dnmax))
    dnavgim = np.array(Image.open(dnavg))

    return nmaxim, dnmaxim, navgim, dnavgim

In [ ]:
nrows =widgets.BoundedIntText(
    value=1,
    min=1,
    max=3,
    step=1,
    description='# plot rows:',
    disabled=False
)
ncols =widgets.BoundedIntText(
    value=1,
    min=1,
    max=3,
    step=1,
    description='# plot columns:',
    disabled=False
)
plot_layout_button = widgets.Button(
    description='Update',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
rowcol = widgets.HBox([nrows, ncols, plot_layout_button])
display(rowcol)

In [2]:
fig, axes = plt.subplots(1, 1, clear=True, sharex=True, sharey=True)
#def on_change(change):
#    global fig
#    global axes
#    fig, axes = plt.subplots(nrows.value, ncols.value, clear=True, sharex=True, sharey=True)
#plot_layout_button.on_click(on_change)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …